### Them cac thu vien can thiet

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType
from pyspark.sql.functions import desc, asc, year
from keras import models, layers
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Khoi tao Spark session

In [2]:
conf = SparkConf().setMaster("local") \
        .setAppName("StockPrediction")

spark = SparkSession \
        .builder \
        .config(conf=conf) \
        .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

### Nap du lieu vao va loai bo cac truong bi null

In [3]:
df = spark.read.format("json") \
        .option("header", "false") \
        .option("inferSchema", "true") \
        .load("./TSLA.json").na.drop()

In [4]:
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- time: string (nullable = true)
 |-- volume: long (nullable = true)



### Tach bo du lieu thanh 2 tap train va test

In [5]:
df = df.withColumn('time', f.to_date('time'))
df = df.withColumn('year', year("time"))

In [6]:
training_set = df.select("time", "open").where(df["year"] < 2020)

In [7]:
test_set = df.select("time", "open").where(df["year"] >=  2020)

In [8]:
df1 = df.toPandas()

In [9]:
df1

,close,high,low,open,time,volume,year
0,61.709999,62.759998,61.549999,62.230000,2015-12-28,782216,2015
1,63.049999,63.540001,62.279999,62.400002,2015-12-29,1417901,2015
2,62.580002,63.130001,62.299999,62.740002,2015-12-30,1178738,2015
3,60.860001,61.869999,60.160000,61.150002,2016-01-04,2885672,2016
4,62.090000,62.310001,60.610001,61.880001,2016-01-05,3154541,2016
...,...,...,...,...,...,...,...
1272,89.000000,90.900002,88.940002,89.620003,2020-12-18,3297694,2020
1273,86.540001,86.959999,82.750000,85.000000,2020-12-21,3398767,2020
1274,87.940002,88.000000,85.510002,87.370003,2020-12-22,1691985,2020
1275,91.949997,92.089996,87.620003,88.000000,2020-12-23,1561928,2020


In [10]:
training_set = training_set.toPandas().values[:,1]

In [11]:
test_set = test_set.toPandas().values[:,1]

In [12]:
open_price = df.select("time", "open").toPandas().values[:,1]

### Xay dung mo hinh du doan bang LSTM

In [13]:
scale = MinMaxScaler()

In [14]:
training_set_scaled = scale.fit_transform(open_price.reshape(-1, 1))
LOOKBACK = 24
X_train = []
y_train = []
for i in range(LOOKBACK, training_set.shape[0]):
    X_train.append(training_set_scaled[i-LOOKBACK:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [15]:
model = Sequential()
model.add(LSTM(units = 12, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 12, return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(units = 1))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
32/32 [==============================] - 2s 66ms/step - loss: 0.3230 - accuracy: 9.9800e-04
Epoch 2/100
32/32 [==============================] - 2s 57ms/step - loss: 0.1199 - accuracy: 9.9800e-04
Epoch 3/100
32/32 [==============================] - 2s 58ms/step - loss: 0.0808 - accuracy: 9.9800e-04
Epoch 4/100
32/32 [==============================] - 2s 56ms/step - loss: 0.0778 - accuracy: 9.9800e-04
Epoch 5/100
32/32 [==============================] - 2s 59ms/step - loss: 0.0716 - accuracy: 9.9800e-04
Epoch 6/100
32/32 [==============================] - 2s 58ms/step - loss: 0.0695 - accuracy: 9.9800e-04
Epoch 7/100
32/32 [==============================] - 2s 59ms/step - loss: 0.0624 - accuracy: 9.9800e-04
Epoch 8/100
32/32 [==============================] - 2s 67ms/step - loss: 0.0613 - accuracy: 9.9800e-04
Epoch 9/100
32/32 [==============================] - 2s 58ms/step - loss: 0.0651 - accuracy: 9.9800e-04
Epoch 10/100
32/32 [==============================] - 2s 62ms/st

32/32 [==============================] - 2s 64ms/step - loss: 0.0382 - accuracy: 9.9800e-04
Epoch 80/100
32/32 [==============================] - 2s 60ms/step - loss: 0.0383 - accuracy: 9.9800e-04
Epoch 81/100
32/32 [==============================] - 2s 70ms/step - loss: 0.0373 - accuracy: 9.9800e-04
Epoch 82/100
32/32 [==============================] - 2s 64ms/step - loss: 0.0381 - accuracy: 9.9800e-04
Epoch 83/100
32/32 [==============================] - 2s 65ms/step - loss: 0.0377 - accuracy: 9.9800e-04
Epoch 84/100
32/32 [==============================] - 2s 64ms/step - loss: 0.0386 - accuracy: 9.9800e-04
Epoch 85/100
32/32 [==============================] - 2s 63ms/step - loss: 0.0380 - accuracy: 9.9800e-04
Epoch 86/100
32/32 [==============================] - 2s 62ms/step - loss: 0.0381 - accuracy: 9.9800e-04
Epoch 87/100
32/32 [==============================] - 2s 63ms/step - loss: 0.0375 - accuracy: 9.9800e-04
Epoch 88/100
32/32 [==============================] - 2s 64ms/step -

In [16]:
total_len = open_price.shape[0]
actual_values = open_price[LOOKBACK:]
inputs = open_price.reshape(-1,1)
inputs = scale.transform(inputs)
X_test = []
for i in range(LOOKBACK, total_len):
    X_test.append(inputs[i-LOOKBACK:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [17]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = scale.inverse_transform(predicted_stock_price)

In [18]:
predicted_stock_price

array([[58.876266],
       [59.08057 ],
       [59.04069 ],
       ...,
       [91.05665 ],
       [89.88813 ],
       [89.11375 ]], dtype=float32)

In [22]:
pd.DataFrame(predicted_stock_price).to_csv("predict.csv")

In [23]:
spark.sparkContext.stop()